In [1]:
import ctypes
libmapx = ctypes.CDLL("/Users/brodzik/lib/libmapx.dylib", ctypes.RTLD_GLOBAL)

In [2]:
libmapx

<CDLL '/Users/brodzik/lib/libmapx.dylib', handle 103aa6350 at 1045b1890>

In [3]:
class GRID (ctypes.Structure) :
    _fields_ = [("map_origin_col", ctypes.c_double),
		("map_origin_row", ctypes.c_double),
		("cols_per_map_unit", ctypes.c_double),
		("rows_per_map_unit", ctypes.c_double),
		("u_min", ctypes.c_double),
		("u_max", ctypes.c_double),
		("cols", ctypes.c_int),
		("rows", ctypes.c_int),
		("gpd_file_ptr", ctypes.c_void_p),
		("gpd_filename", ctypes.c_char_p),
		("mapx_class", ctypes.c_void_p)     ]


In [4]:
libmapx.init_grid.restype = ctypes.POINTER(GRID)

In [7]:
grid_p = libmapx.init_grid("/Users/brodzik/lib/maps/EASE2_M25km.gpd")
grid = libmapx.init_grid("/Users/brodzik/lib/maps/EASE2_M25km.gpd")[0]

In [8]:
print grid.u_min, grid.map_origin_row, grid.cols, grid.rows

0.0 -0.5 1388 584


In [23]:
libmapx.forward_grid.argtypes = [ctypes.POINTER(GRID), ctypes.c_double, ctypes.c_double,
				 ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double)]

In [28]:
grid
lat, lon, r, s = ctypes.c_double(25.0), ctypes.c_double(0.0), ctypes.c_double(0.0), ctypes.c_double(0.0)
result = libmapx.forward_grid(grid, lat, lon, ctypes.byref(r), ctypes.byref(s))

In [29]:
print result, lat.value, lon.value, r.value, s.value

1 25.0 0.0 693.5 167.962599743


In [30]:
libmapx.inverse_grid.argtypes = [ctypes.POINTER(GRID),
				 ctypes.c_double, ctypes.c_double,
				 ctypes.POINTER(ctypes.c_double),
				 ctypes.POINTER(ctypes.c_double)  ]
inv = libmapx.inverse_grid(grid, r, s, ctypes.byref(lat), ctypes.byref(lon))
print inv, r.value, s.value, lat.value, lon.value

1 693.5 167.962599743 24.9999999864 0.0


In [33]:
print grid_p[0].gpd_filename, grid_p[0].gpd_file_ptr
lat = ctypes.c_double(-25.0)
lon = ctypes.c_double(0.0)
r = ctypes.c_double(0.0)
s = ctypes.c_double(0.0)
print lat.value, lon.value, r.value, s.value
libmapx.forward_grid(grid_p, lat, lon, ctypes.byref(r), ctypes.byref(s))
print lat.value, lon.value, r.value, s.value

/Users/brodzik/lib/maps/EASE2_M25km.gpd 140735238571264
-25.0 0.0 0.0 0.0
-25.0 0.0 693.5 415.037400257


In [34]:
class FGRID(ctypes.Structure):
    _fields_ = [("grid_p", ctypes.POINTER(GRID)),
		("lat", ctypes.c_double),
		("lon", ctypes.c_double),
		("r", ctypes.c_double),
		("s", ctypes.c_double)             ]
class RGRID(ctypes.Structure):
    _fields_ = [("grid_p", ctypes.POINTER(GRID)),
		("r", ctypes.c_double),
		("s", ctypes.c_double),
		("lat", ctypes.c_double),
		("lon", ctypes.c_double)             ]


In [35]:
libmapx.forward_grid.argtypes = [ctypes.POINTER(FGRID)]
libmapx.inverse_grid.argtypes = [ctypes.POINTER(RGRID)]

In [36]:
fgrid = FGRID() 
fgrid.lon = 0.0
fgrid.lat = 0.0
fgrid.grid_p = grid_p


In [37]:
print grid.u_min

0.0
